## Merge all datasets
Arctic dataframe and dataframe from Waterhouse et al., made in data_exploration/load_ds notebook

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [13]:
import sys
import os
sys.path.append('../..')
from src.utils.directories import get_parent_directory

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [14]:
parent_dir = get_parent_directory()

In [15]:
parent_dir

'/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence'

In [16]:
global_pkl = os.path.join(parent_dir, "data/processed/Mashayek2022/input_microstructure.pkl")
global_df = pd.read_pickle(global_pkl)
global_df = global_df.rename(columns={'lat': 'latitude', 'log_N2_sort': 'log_N2'})

# Rename columns using the rename() method
waterhouse_df = global_df.rename(columns={'log_N2_sort': 'log_N2', 'log_kappa': 'LK', 'lat': 'latitude'})

In [17]:
# arctic_pkl = '/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/processed_data/ml_ready/arctic2_1805.pkl'
arctic_pkl = (os.path.join(parent_dir, "data/processed/ml_ready/1710_time.pkl"))
arctic_df = pd.read_pickle(arctic_pkl)

## Concatenate dataframes

In [18]:
all_df = pd.concat([arctic_df, waterhouse_df], axis=0)
arctic_df = all_df[['depth', 'profile', 'cruise', 'latitude', 'longitude',
                 'S', 'T', 'log_eps', 'log_N2', 'dTdz', 'dSdz', 'time', 'season', 'seasonal_sin']]
all_df = all_df[['depth', 'profile', 'cruise', 'latitude',
                 'S', 'T', 'log_eps', 'log_N2', 'dTdz', 'dSdz']]

## Save merged dataframe

In [21]:
arctic_pkl_path = (os.path.join(parent_dir, "data/processed/ml_ready/merged_arctic.pkl"))
arctic_df.to_pickle(arctic_pkl_path)

all_pkl_path = (os.path.join(parent_dir, "data/processed/ml_ready/merged_all.pkl"))
all_df.to_pickle(all_pkl_path)